In [ ]:
import threading
import time
import random
import requests
import logging
import json
import os
import hashlib
import hmac
import base64
from datetime import datetime, timedelta
from typing import Dict, List, Tuple, Optional
import uuid

import flask
from flask import Flask, request, jsonify, g
from flask_limiter import Limiter
from flask_limiter.util import get_remote_address
from werkzeug.security import generate_password_hash, check_password_hash
import jwt
from cryptography.fernet import Fernet

# Generate encryption key (in production, store securely)
ENCRYPTION_KEY = os.getenv('ENCRYPTION_KEY', Fernet.generate_key())
cipher_suite = Fernet(ENCRYPTION_KEY)

# JWT Secret (in production, store in secure environment)
JWT_SECRET = os.getenv('JWT_SECRET', os.urandom(32).hex())

# Set up comprehensive logging with security events
logging.basicConfig(
    level=logging.INFO,
    format='%(asctime)s - %(levelname)s - [%(name)s] [%(filename)s:%(lineno)d] - %(message)s',
    handlers=[
        logging.FileHandler("healthcare_monitor.log"),
        logging.StreamHandler()
    ]
)
logger = logging.getLogger("HealthcareMonitor")

# Configuration Parameters for vital sign thresholds
VITAL_SIGN_RANGES = {
    "heart_rate": (50, 120),          # Acceptable heart rate range (beats per minute)
    "blood_pressure_systolic": (90, 140),  # Acceptable systolic blood pressure (mmHg)
    "blood_pressure_diastolic": (60, 90),  # Acceptable diastolic blood pressure (mmHg)
    "oxygen_saturation": (95, 100)    # Acceptable oxygen saturation (%)
}

# Simulated user database (In production, use a secure database with proper password hashing)
USERS = {
    "doctor": {
        "password": generate_password_hash("secure_password_1"),
        "role": "doctor"
    },
    "nurse": {
        "password": generate_password_hash("secure_password_2"),
        "role": "nurse"
    },
    "admin": {
        "password": generate_password_hash("secure_admin_password"),
        "role": "admin"
    }
}

# In-memory store for patient data (in production, use a secure database)
patient_data_store = {}

In [ ]:
# Security functions
def encrypt_data(data: str) -> bytes:
    """Encrypt sensitive data before storage"""
    if isinstance(data, dict):
        data = json.dumps(data)
    return cipher_suite.encrypt(data.encode())

def decrypt_data(encrypted_data: bytes) -> str:
    """Decrypt data for processing"""
    try:
        return cipher_suite.decrypt(encrypted_data).decode()
    except Exception as e:
        logger.error(f"Decryption error: {str(e)}")
        raise

def generate_token(username: str, role: str) -> str:
    """Generate a JWT token for the authenticated user"""
    expiration = datetime.utcnow() + timedelta(hours=1)
    payload = {
        "sub": username,
        "role": role,
        "iat": datetime.utcnow(),
        "exp": expiration,
        "jti": str(uuid.uuid4())  # Add unique token ID to prevent replay attacks
    }
    return jwt.encode(payload, JWT_SECRET, algorithm="HS256")

def validate_token(token: str) -> Optional[Dict]:
    """Validate JWT token and extract payload"""
    try:
        payload = jwt.decode(token, JWT_SECRET, algorithms=["HS256"])
        return payload
    except jwt.ExpiredSignatureError:
        logger.warning("Authentication attempt with expired token")
        return None
    except jwt.InvalidTokenError:
        logger.warning("Authentication attempt with invalid token")
        return None

def requires_auth(role_required=None):
    """Decorator for endpoints that require authentication"""
    def decorator(f):
        def wrapped(*args, **kwargs):
            auth_header = request.headers.get('Authorization')

            if not auth_header or not auth_header.startswith('Bearer '):
                logger.warning(f"Unauthorized access attempt to {request.path} without proper Authorization header")
                return jsonify({"error": "Authentication required"}), 401

            token = auth_header.split(' ')[1]
            payload = validate_token(token)

            if not payload:
                logger.warning(f"Invalid token used to access {request.path}")
                return jsonify({"error": "Invalid or expired token"}), 401

            if role_required and payload.get('role') != role_required:
                logger.warning(f"Access denied: User with role {payload.get('role')} attempted to access {request.path} requiring role {role_required}")
                return jsonify({"error": "Insufficient privileges"}), 403

            # Store user info for the request
            g.user = payload
            return f(*args, **kwargs)

        wrapped.__name__ = f.__name__
        return wrapped

    return decorator

def validate_patient_data(data: Dict) -> Tuple[bool, str]:
    """Validate incoming patient data structure and types"""
    # Check for required fields
    required_fields = ['patient_id', 'heart_rate', 'blood_pressure_systolic',
                      'blood_pressure_diastolic', 'oxygen_saturation']

    missing_fields = [field for field in required_fields if field not in data]
    if missing_fields:
        return False, f"Missing required fields: {', '.join(missing_fields)}"

    # Validate data types
    if not isinstance(data.get('patient_id'), str):
        return False, "Patient ID must be a string"

    # Validate numeric fields
    numeric_fields = ['heart_rate', 'blood_pressure_systolic',
                     'blood_pressure_diastolic', 'oxygen_saturation']

    for field in numeric_fields:
        value = data.get(field)
        if not isinstance(value, (int, float)):
            return False, f"{field} must be a number"

        # Prevent unrealistic values (basic sanity checks)
        if field == 'heart_rate' and (value < 20 or value > 250):
            return False, f"Heart rate value {value} is outside realistic range"
        elif field == 'blood_pressure_systolic' and (value < 50 or value > 250):
            return False, f"Systolic blood pressure value {value} is outside realistic range"
        elif field == 'blood_pressure_diastolic' and (value < 30 or value > 150):
            return False, f"Diastolic blood pressure value {value} is outside realistic range"
        elif field == 'oxygen_saturation' and (value < 50 or value > 100):
            return False, f"Oxygen saturation value {value} is outside realistic range"

    return True, "Validation successful"

def detect_anomalies(data: Dict) -> List[str]:
    """Enhanced anomaly detection for vital signs"""
    anomalies = []

    # Check vital signs against threshold ranges
    if data['heart_rate'] < VITAL_SIGN_RANGES['heart_rate'][0]:
        anomalies.append(f"Low heart rate detected: {data['heart_rate']} bpm")
    elif data['heart_rate'] > VITAL_SIGN_RANGES['heart_rate'][1]:
        anomalies.append(f"High heart rate detected: {data['heart_rate']} bpm")

    if data['blood_pressure_systolic'] < VITAL_SIGN_RANGES['blood_pressure_systolic'][0]:
        anomalies.append(f"Low systolic blood pressure detected: {data['blood_pressure_systolic']} mmHg")
    elif data['blood_pressure_systolic'] > VITAL_SIGN_RANGES['blood_pressure_systolic'][1]:
        anomalies.append(f"High systolic blood pressure detected: {data['blood_pressure_systolic']} mmHg")

    if data['blood_pressure_diastolic'] < VITAL_SIGN_RANGES['blood_pressure_diastolic'][0]:
        anomalies.append(f"Low diastolic blood pressure detected: {data['blood_pressure_diastolic']} mmHg")
    elif data['blood_pressure_diastolic'] > VITAL_SIGN_RANGES['blood_pressure_diastolic'][1]:
        anomalies.append(f"High diastolic blood pressure detected: {data['blood_pressure_diastolic']} mmHg")

    if data['oxygen_saturation'] < VITAL_SIGN_RANGES['oxygen_saturation'][0]:
        anomalies.append(f"Low oxygen saturation detected: {data['oxygen_saturation']}%")

    # Check for rapid changes if we have historical data
    patient_id = data['patient_id']
    if patient_id in patient_data_store and len(patient_data_store[patient_id]) > 0:
        # Get the most recent reading
        latest_data = json.loads(decrypt_data(patient_data_store[patient_id][-1]))

        # Check for rapid heart rate change (more than 20% change)
        hr_change_pct = abs(data['heart_rate'] - latest_data['heart_rate']) / latest_data['heart_rate'] * 100
        if hr_change_pct > 20:
            anomalies.append(f"Rapid heart rate change: {hr_change_pct:.1f}% difference")

        # Check for rapid blood pressure change
        sys_change_pct = abs(data['blood_pressure_systolic'] - latest_data['blood_pressure_systolic']) / latest_data['blood_pressure_systolic'] * 100
        if sys_change_pct > 15:
            anomalies.append(f"Rapid systolic blood pressure change: {sys_change_pct:.1f}% difference")

    return anomalies

In [ ]:
# Initialize Flask app
app = Flask(__name__)
app.config['JSON_SORT_KEYS'] = False  # Preserve order in JSON responses

# Configure rate limiting to prevent brute force and DoS attacks
limiter = Limiter(
    key_func=get_remote_address,
    app=app,
    default_limits=["200 per day", "50 per hour"],
    storage_uri="memory://"
)

def add_security_headers(response):
    """Add security headers to HTTP responses"""
    response.headers['X-Content-Type-Options'] = 'nosniff'
    response.headers['X-Frame-Options'] = 'DENY'
    response.headers['X-XSS-Protection'] = '1; mode=block'
    response.headers['Content-Security-Policy'] = "default-src 'self'"
    response.headers['Strict-Transport-Security'] = 'max-age=31536000; includeSubDomains'
    return response

# Register the security headers function
@app.after_request
def after_request(response):
    return add_security_headers(response)

# Error handlers
@app.errorhandler(404)
def not_found(error):
    return jsonify({"error": "Resource not found"}), 404

@app.errorhandler(500)
def server_error(error):
    logger.error(f"Internal server error: {error}")
    return jsonify({"error": "Internal server error"}), 500

In [ ]:
# Authentication endpoint
@app.route('/api/auth', methods=['POST'])
@limiter.limit("5 per minute")  # Limit login attempts
def authenticate():
    """Authenticate users and provide JWT token"""
    if not request.is_json:
        return jsonify({"error": "Request must be JSON"}), 400

    data = request.get_json()

    # Validate input
    if not data or not data.get('username') or not data.get('password'):
        logger.warning("Login attempt with missing credentials")
        return jsonify({"error": "Username and password required"}), 400

    username = data.get('username')
    password = data.get('password')

    # Prevent timing attacks by using constant time comparison
    if username not in USERS or not check_password_hash(USERS[username]['password'], password):
        logger.warning(f"Failed login attempt for username: {username}")
        time.sleep(1)  # Add delay to prevent brute force
        return jsonify({"error": "Invalid credentials"}), 401

    # Generate JWT token
    token = generate_token(username, USERS[username]['role'])
    logger.info(f"Successful authentication for user: {username}")

    return jsonify({
        "token": token,
        "role": USERS[username]['role'],
        "expires_in": 3600  # Token expiration in seconds
    }), 200

In [ ]:
# Main endpoint for receiving patient data
@app.route('/api/patient-data', methods=['POST'])
@requires_auth()  # Any authenticated user can submit data
def patient_data():
    """Receive and process patient vital signs data"""
    # Validate JSON format
    if not request.is_json:
        logger.warning("Non-JSON data received at patient-data endpoint")
        return jsonify({"error": "Request must be JSON format"}), 400

    data = request.get_json()

    # Validate data structure and types
    is_valid, validation_message = validate_patient_data(data)
    if not is_valid:
        logger.warning(f"Invalid patient data received: {validation_message}")
        return jsonify({"error": validation_message}), 400

    patient_id = data.get('patient_id')

    # Add timestamp if not present
    if 'timestamp' not in data:
        data['timestamp'] = datetime.utcnow().isoformat()

    # Detect anomalies in vital signs
    anomalies = detect_anomalies(data)

    # Store encrypted data
    encrypted_data = encrypt_data(json.dumps(data))
    if patient_id not in patient_data_store:
        patient_data_store[patient_id] = []

    # Limit storage to last 100 readings per patient (in production, use proper database)
    if len(patient_data_store[patient_id]) >= 100:
        patient_data_store[patient_id].pop(0)

    patient_data_store[patient_id].append(encrypted_data)

    # Log any detected anomalies
    if anomalies:
        logger.warning(f"Anomalies detected for patient {patient_id}: {anomalies}")
        # In a real system, this could trigger alerts to medical staff

    # Create audit log entry
    logger.info(f"Data received from Patient {patient_id} by user {g.user['sub']} (role: {g.user['role']})")

    return jsonify({
        "status": "success",
        "patient_id": patient_id,
        "timestamp": data['timestamp'],
        "anomalies": anomalies
    }), 200

In [ ]:
# Endpoint to retrieve patient history - restricted to doctors
@app.route('/api/patient/<patient_id>/history', methods=['GET'])
@requires_auth('doctor')
def get_patient_history(patient_id):
    """Retrieve historical data for a specific patient"""
    if patient_id not in patient_data_store:
        return jsonify({"error": "Patient not found"}), 404

    # Get optional limit parameter with a default of 10
    try:
        limit = min(int(request.args.get('limit', 10)), 100)
    except ValueError:
        return jsonify({"error": "Invalid limit parameter"}), 400

    # Decrypt and return the last 'limit' records
    try:
        records = patient_data_store[patient_id][-limit:]
        history = [json.loads(decrypt_data(record)) for record in records]

        logger.info(f"Patient history accessed for {patient_id} by {g.user['sub']} (role: {g.user['role']})")

        return jsonify({
            "patient_id": patient_id,
            "count": len(history),
            "history": history
        }), 200
    except Exception as e:
        logger.error(f"Error retrieving patient history: {str(e)}")
        return jsonify({"error": "Error retrieving patient history"}), 500

# Admin endpoint to manage system configuration
@app.route('/api/admin/configuration', methods=['GET', 'PUT'])
@requires_auth('admin')
def manage_configuration():
    """Get or update system configuration - admin only"""
    if request.method == 'GET':
        # Return current configuration (excluding sensitive data)
        config = {
            "vital_sign_ranges": VITAL_SIGN_RANGES,
            "version": "1.0.0",
            "log_level": logging.getLevelName(logger.level)
        }
        return jsonify(config), 200

    elif request.method == 'PUT':
        # Update configuration
        if not request.is_json:
            return jsonify({"error": "Request must be JSON"}), 400

        data = request.get_json()
        updated = []

        # Update vital sign ranges if provided
        if 'vital_sign_ranges' in data:
            try:
                for key, value in data['vital_sign_ranges'].items():
                    if key in VITAL_SIGN_RANGES and isinstance(value, (list, tuple)) and len(value) == 2:
                        # Validate range values are reasonable
                        if all(isinstance(v, (int, float)) for v in value) and value[0] < value[1]:
                            VITAL_SIGN_RANGES[key] = (value[0], value[1])
                            updated.append(key)
                        else:
                            return jsonify({"error": f"Invalid range values for {key}"}), 400
            except Exception as e:
                logger.error(f"Error updating vital sign ranges: {str(e)}")
                return jsonify({"error": "Error updating configuration"}), 500

        # Update log level if provided
        if 'log_level' in data:
            try:
                level = data['log_level'].upper()
                if level in ('DEBUG', 'INFO', 'WARNING', 'ERROR', 'CRITICAL'):
                    logger.setLevel(level)
                    updated.append('log_level')
                else:
                    return jsonify({"error": f"Invalid log level: {level}"}), 400
            except Exception as e:
                logger.error(f"Error updating log level: {str(e)}")
                return jsonify({"error": "Error updating log level"}), 500

        logger.info(f"Configuration updated by {g.user['sub']}: {', '.join(updated)}")

        return jsonify({
            "status": "success",
            "message": "Configuration updated successfully",
            "updated": updated
        }), 200

# Health check endpoint (no auth required)
@app.route('/api/health', methods=['GET'])
def health_check():
    """Basic health check endpoint"""
    return jsonify({
        "status": "operational",
        "timestamp": datetime.utcnow().isoformat()
    }), 200

In [ ]:
# Sensor simulator class
class SecurePatientSensor(threading.Thread):
    """Secure patient sensor simulator with TLS support"""
    def __init__(self, patient_id, server_url, auth_token, simulate_anomalies=True):
        threading.Thread.__init__(self)
        self.patient_id = patient_id
        self.server_url = server_url
        self.auth_token = auth_token
        self.simulate_anomalies = simulate_anomalies
        self.anomaly_probability = 0.2  # 20% chance of anomaly
        self.running = True
        self.daemon = True

    def generate_vital_signs(self):
        """Generate simulated vital sign data with occasional anomalies"""
        # Decide if this reading should have an anomaly
        generate_anomaly = self.simulate_anomalies and random.random() < self.anomaly_probability

        if generate_anomaly:
            # Choose which vital sign will have an anomaly
            anomaly_type = random.choice(['heart_rate', 'blood_pressure', 'oxygen'])

            if anomaly_type == 'heart_rate':
                # Generate abnormal heart rate (too low or too high)
                heart_rate = random.choice([
                    random.uniform(30, 45),  # Too low
                    random.uniform(130, 180)  # Too high
                ])
                # Normal values for other signs
                blood_pressure_systolic = random.uniform(90, 130)
                blood_pressure_diastolic = random.uniform(60, 85)
                oxygen_saturation = random.uniform(95, 99)

            elif anomaly_type == 'blood_pressure':
                # Normal heart rate
                heart_rate = random.uniform(60, 100)
                # Abnormal blood pressure (various possibilities)
                if random.choice([True, False]):
                    # High blood pressure
                    blood_pressure_systolic = random.uniform(150, 200)
                    blood_pressure_diastolic = random.uniform(90, 110)
                else:
                    # Low blood pressure
                    blood_pressure_systolic = random.uniform(70, 85)
                    blood_pressure_diastolic = random.uniform(40, 55)
                # Normal oxygen
                oxygen_saturation = random.uniform(95, 99)

            else:  # oxygen anomaly
                # Normal heart rate and blood pressure
                heart_rate = random.uniform(60, 100)
                blood_pressure_systolic = random.uniform(90, 130)
                blood_pressure_diastolic = random.uniform(60, 85)
                # Low oxygen
                oxygen_saturation = random.uniform(85, 94)
        else:
            # Generate all normal vital signs
            heart_rate = random.uniform(60, 100)
            blood_pressure_systolic = random.uniform(90, 130)
            blood_pressure_diastolic = random.uniform(60, 85)
            oxygen_saturation = random.uniform(95, 99)

        # Create the data packet
        return {
            "patient_id": self.patient_id,
            "heart_rate": round(heart_rate, 1),
            "blood_pressure_systolic": round(blood_pressure_systolic, 1),
            "blood_pressure_diastolic": round(blood_pressure_diastolic, 1),
            "oxygen_saturation": round(oxygen_saturation, 1),
            "timestamp": datetime.utcnow().isoformat()
        }

    def run(self):
        """Run the secure sensor simulation"""
        logger.info(f"Starting secure sensor simulation for patient {self.patient_id}")

        while self.running:
            try:
                # Generate vital sign data
                vital_data = self.generate_vital_signs()

                # Add proper authentication and headers
                headers = {
                    'Content-Type': 'application/json',
                    'Authorization': f'Bearer {self.auth_token}',
                    'User-Agent': f'HealthSensor/{self.patient_id}'
                }

                # Send data securely with proper error handling
                response = requests.post(
                    self.server_url,
                    json=vital_data,
                    headers=headers,
                    timeout=5,  # 5 second timeout
                    verify=True  # Verify SSL certificates in production
                )

                if response.status_code == 200:
                    result = response.json()
                    if result.get('anomalies'):
                        logger.warning(f"Patient {self.patient_id}: Anomalies detected - {result['anomalies']}")
                    else:
                        logger.info(f"Patient {self.patient_id}: Data sent successfully")
                else:
                    logger.error(f"Patient {self.patient_id}: Failed to send data - HTTP {response.status_code}")

            except requests.exceptions.RequestException as e:
                logger.error(f"Patient {self.patient_id}: Connection error - {str(e)}")
            except Exception as e:
                logger.error(f"Patient {self.patient_id}: Error in sensor simulation - {str(e)}")

            # Random interval between readings (8-12 seconds)
            time.sleep(random.uniform(8, 12))

    def stop(self):
        """Stop the sensor simulation"""
        self.running = False
        logger.info(f"Stopping sensor simulation for patient {self.patient_id}")

In [ ]:
# Main function to start the system
def main():
    """Initialize and start the healthcare monitoring system"""
    logger.info("Starting Healthcare Monitoring System...")

    # Start the Flask server in a separate thread
    server_thread = threading.Thread(target=lambda: app.run(host="0.0.0.0", port=5000, debug=False, use_reloader=False))
    server_thread.daemon = True
    server_thread.start()
    logger.info("Flask server started in background thread.")

    # Wait for server to start
    time.sleep(2)

    try:
        # Authenticate to get a valid token
        auth_data = {
            "username": "doctor",
            "password": "secure_password_1"
        }

        auth_response = requests.post(
            "http://localhost:5000/api/auth",
            json=auth_data,
            timeout=5
        )

        if auth_response.status_code != 200:
            logger.error(f"Authentication failed: {auth_response.status_code}")
            return

        token = auth_response.json()['token']
        logger.info("Authentication successful, starting sensor simulations...")

        # Start multiple patient simulations
        sensors = []
        for i in range(1, 4):  # 3 patients
            patient_id = f"PATIENT_{i:03d}"
            sensor = SecurePatientSensor(
                patient_id=patient_id,
                server_url="http://localhost:5000/api/patient-data",
                auth_token=token
            )
            sensor.start()
            sensors.append(sensor)
            logger.info(f"Started secure sensor simulation for {patient_id}")

        # Keep the main thread alive
        try:
            while True:
                time.sleep(1)
        except KeyboardInterrupt:
            logger.info("Shutdown initiated...")

            # Stop all sensor simulations
            for sensor in sensors:
                sensor.stop()

            # Wait for all sensors to stop
            for sensor in sensors:
                sensor.join(timeout=2)

            logger.info("Healthcare Monitoring System shutdown complete.")

    except Exception as e:
        logger.error(f"Error in main execution: {str(e)}")

In [ ]:
# Run the application
if __name__ == "__main__":
    main()

 * Serving Flask app '__main__'
 * Debug mode: off


INFO:werkzeug:WARNING: This is a development server. Do not use it in a production deployment. Use a production WSGI server instead.
 * Running on all addresses (0.0.0.0)
 * Running on http://127.0.0.1:5000
 * Running on http://172.28.0.12:5000
INFO:werkzeug:Press CTRL+C to quit
INFO:werkzeug:127.0.0.1 - - [02/Apr/2025 01:16:26] "POST /api/auth HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [02/Apr/2025 01:16:26] "POST /api/patient-data HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [02/Apr/2025 01:16:26] "POST /api/patient-data HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [02/Apr/2025 01:16:26] "POST /api/patient-data HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [02/Apr/2025 01:16:34] "POST /api/patient-data HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [02/Apr/2025 01:16:34] "POST /api/patient-data HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [02/Apr/2025 01:16:35] "POST /api/patient-data HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [02/Apr/2025 01:16:42] "POST /api/patient-data HTTP/1.1" 200 -
